In [1]:
%reload_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from functools import reduce
from datetime import timedelta, date, datetime
from pytz import timezone

* 3, 4 - oven (draws voltage from both 120 volt legs)
* 5 - refrigerator 
* 6 - dishwasher
* 7, 8, 15, 16 - kitchen_outlets 
* 9, 17, 18 - lighting 
* 10, 20 - washer_dryer (10 is the motor and 20 is the electric heating element)
* 11 - microwave
* 12 - bathroom ground fault interpreter
* 13 - electric space heater
* 14 - electric stove
* 19 - not a washer dryer definitely


In [3]:
house_num =1 #5#1, 2, 6 

In [7]:
labels = get_labels(house_num);

In [8]:
def get_labels(house_num, PATH='data/low_freq/'): #reads the house_number's labels.dat and returns a dataframe
    '''
    input:
        house_num (int)
        channel_num (int)
    return:
        pandas.core.frame.DataFrame
    '''
    return pd.read_table("{0}house_{1}/labels.dat".format(PATH, house_num), header=None, delim_whitespace=True,
                         names=['channel', 'appliance'], dtype={'channel': np.int16, 'appliance': np.str})

def get_channel_data(channel_num, house_num, as_csv=False):
    '''
    input: 
        labels (List[int])
        channel_num (int)
        house_num (int)
        as_csv (bool)
    return:
        True if as_csv=True
        pandas.core.frame.DataFrame if as_csv=False
    '''
    PATH = "data/low_freq/house_{0}/channel_{1}".format(house_num, channel_num)+'.dat'
    labels = get_labels(house_num)
    if as_csv:
        ret_file = labels.loc[labels['channel'] == channel_num].iloc[0][1]
        pd.read_table(PATH, header=None, delim_whitespace=True,
                        names=['time_stamp', 'power'],
                             dtype={'time_stamp': np.string_, 'power': np.float128}).to_csv(ret_file+'_data.csv')
        return True
    else:
        return pd.read_table(PATH, header=None, delim_whitespace=True,
                        names=['time_stamp', 'power_{0}'.format(labels.loc[labels['channel'] == channel_num].iloc[0][1])],
                             dtype={'time_stamp': np.string_, 'power': np.float128})


def merge_all(ch_nums, house_num):
    full_data = get_channel_data(ch_nums[0], house_num)
    for ch in ch_nums[1:]:
        full_data = full_data.merge(get_channel_data(ch, house_num), how='inner', on=['time_stamp'])
    return full_data


#to_csv function
def to_csv(df_list, house_num):
    for df in df_list:
        df.to_csv('data/house_{0}/hour_by_hour/'.format(house_num) + str(df.name) + '.csv') #change this to the corresponding house

def daterange(start, end):
    for n in range(int((end - start).days)):
        yield start + timedelta(n)

def grp_to_csv(grpby, day, house_num):
    tdf = pd.DataFrame(grpby)
    tdf.index = tdf.index.map(lambda x: pd.to_datetime(x).hour)
    tdf.to_csv('data/house_{0}/dayhour/day_'.format(house_num) + '{0}'.format(day) + '.csv')

def get_dayhour(df, start_date, end_date, house_num):
    day = 1
    for d in daterange(start_date, end_date):
        grp_to_csv(df.groupby(pd.Grouper(freq='D')).get_group(d), day, house_num)
        day += 1

* 3, 4 - oven (draws voltage from both 120 volt legs)
* 5 - refrigerator 
* 6 - dishwasher
* 7, 8, 15, 16 - kitchen_outlets 
* 9, 17, 18 - lighting 
* 10, 20 - washer_dryer (10 is the motor and 20 is the electric heating element)
* 11 - microwave
* 12 - bathroom ground fault interpreter
* 13 - electric space heater
* 14 - electric stove
* 19 - not a washer dryer definitely


In [81]:
tmp = get_channel_data(12, 1) #channel_num, house_num

In [89]:
house_num = 1
ch_nums = [3, 4, 5, 6, 7, 8, 9, 17, 18, 10, 20, 11, 13, 14] #refrigerator, dishwasher, microwave, elec_heat, stove house_1
#ch_nums = [9, 10, 6, 4, 5] #refrigerator, dishwasher, microwave, lighting, stove house_2
#ch_nums = [8, 9, 4, 12, 5] #refrigerator, dishwasher, washer_dryer, elec_heat, stove house_6
full_data = merge_all(ch_nums, house_num) #merges all required channels of a house

In [ ]:
full_data['time_stamp'] = pd.to_datetime(full_data['time_stamp'], unit='s') #converts to datetime

In [90]:
#get_channel_data(15, 1)['power_kitchen_outlets']
full_data['power_kitchen_outlets_x_1'] = get_channel_data(15, 1)['power_kitchen_outlets']
full_data['power_kitchen_outlets_y_1'] = get_channel_data(16, 1)['power_kitchen_outlets']

In [ ]:
print(full_data.keys())
max(full_data['power_kitchen_outlets_x_1'])

In [91]:
fd = pd.DataFrame()
fd['time_stamp'] = full_data['time_stamp']
#fd['oven'] = full_data['power_oven_x'] + full_data['power_oven_x']
fd['refrigerator'] = full_data['power_refrigerator']
fd['dish'] = full_data['power_dishwaser']
fd['kitchen'] = full_data['power_kitchen_outlets_x'] + full_data['power_kitchen_outlets_y'] + full_data['power_kitchen_outlets_x_1'] + full_data['power_kitchen_outlets_y_1'] + full_data['power_oven_x'] + full_data['power_oven_x'] + full_data['power_stove']
fd['lighting'] = full_data['power_lighting'] + full_data['power_lighting_x'] + full_data['power_lighting_y']
fd['washer'] = full_data['power_washer_dryer_x'] + full_data['power_washer_dryer_y']
fd['microwave'] = full_data['power_microwave']
#fd['elecheat'] = full_data['power_electric_heat']
#fd['stove'] = full_data['power_stove']


In [93]:
fd['time_stamp'] = pd.to_datetime(fd['time_stamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert('US/Pacific').apply(lambda x: datetime(x.year, x.month, x.day, x.hour, x.minute, x.second))

In [94]:
fd.head()

,time_stamp,refrigerator,dish,kitchen,lighting,washer,microwave
0,2011-04-18 06:22:13,6.0,0.0,57.0,192.0,0.0,5.0
1,2011-04-18 06:22:16,6.0,0.0,57.0,192.0,0.0,5.0
2,2011-04-18 06:22:20,6.0,0.0,58.0,192.0,0.0,5.0
3,2011-04-18 06:22:23,6.0,1.0,57.0,192.0,0.0,5.0
4,2011-04-18 06:22:26,6.0,0.0,57.0,192.0,0.0,5.0


In [95]:
day_by_day = fd.resample(rule='D', on='time_stamp').sum() #aggregation by day
#day_by_day = full_data.resample(rule='D', on='time_stamp').sum() #aggregation by day

In [96]:
day_by_day.head()

,refrigerator,dish,kitchen,lighting,washer,microwave
time_stamp,,,,,,
2011-04-18,998457.0,1108055.0,1803399.0,1503336.0,1113.5,539775.0
2011-04-19,1116447.0,1198.0,1157629.0,2358940.0,630.0,411167.0
2011-04-20,1097970.0,793.0,1234689.0,1766586.0,36.5,535087.0
2011-04-21,1105265.0,1481.0,1088088.0,1309990.0,187.0,377186.0
2011-04-22,993582.0,792.0,1182754.0,1288460.0,17.5,138326.0


In [97]:
day_by_day.to_csv('data/house_1/day_by_day/day_by_day_house_1.csv') #daybyday for stacked bar

In [98]:
day_by_day.sum(axis=0)

refrigerator    42301545.0
dish            18246649.0
kitchen         50977384.0
lighting        57485182.0
washer          30206552.0
microwave       15679973.0
dtype: float64

In [99]:
hour_by_hour = fd.resample(rule='H', on='time_stamp').sum() #aggregation by hour
#hour_by_hour = full_data.resample(rule='H', on='time_stamp').sum() #aggregation by hour

In [100]:
hour_by_hour.head()

,refrigerator,dish,kitchen,lighting,washer,microwave
time_stamp,,,,,,
2011-04-18 06:00:00,3563.0,113.0,31959.0,98552.0,4.5,2662.0
2011-04-18 07:00:00,134014.0,425.0,42840.0,96306.0,36.0,27448.0
2011-04-18 08:00:00,116762.0,8.0,41431.0,78320.0,2.5,75297.0
2011-04-18 09:00:00,54328.0,10.0,40392.0,45597.0,0.0,3732.0
2011-04-18 10:00:00,46438.0,8.0,39124.0,2409.0,0.0,3620.0


In [101]:
start_date = date(2011, 4, 18) #change
end_date = date(2011, 5, 25) #change
get_dayhour(hour_by_hour, start_date, end_date, house_num) #dayhour for multiseries 

In [102]:
#for heatmap
basedate = datetime(2011, 4, 18, 0, 0, 0)
hh_day_hour = hour_by_hour.copy()
hh_day_hour.insert(loc=0, column='day', value=hh_day_hour.index.map(lambda x: (x.to_datetime() - basedate).days + 1))
hh_day_hour.index = hh_day_hour.index.map(lambda x: pd.to_datetime(x).hour + 1)

In [103]:
hh_day_hour.head()

,day,refrigerator,dish,kitchen,lighting,washer,microwave
time_stamp,,,,,,,
7,1,3563.0,113.0,31959.0,98552.0,4.5,2662.0
8,1,134014.0,425.0,42840.0,96306.0,36.0,27448.0
9,1,116762.0,8.0,41431.0,78320.0,2.5,75297.0
10,1,54328.0,10.0,40392.0,45597.0,0.0,3732.0
11,1,46438.0,8.0,39124.0,2409.0,0.0,3620.0


In [104]:
hour_by_hour.keys()

Index(['refrigerator', 'dish', 'kitchen', 'lighting', 'washer', 'microwave'], dtype='object')

In [105]:
#oven_hh = hh_day_hour[['day', 'oven']]
#oven_hh.name = 'oven_hh'

refrigerator_hh = hh_day_hour[['day', 'refrigerator']]
refrigerator_hh.name = 'refrigerator_hh'

dishwasher_hh = hh_day_hour[['day', 'dish']]
dishwasher_hh.name = 'dishwasher_hh'

kitchen_hh = hh_day_hour[['day', 'kitchen']]
kitchen_hh.name = 'kitchen_hh'

lighting_hh = hh_day_hour[['day', 'lighting']]
lighting_hh.name = 'lighting_hh'

washer_hh = hh_day_hour[['day', 'washer']]
washer_hh.name = 'washer_hh'

microwave_hh = hh_day_hour[['day', 'microwave']]
microwave_hh.name = 'microwave_hh'

#elecheat_hh = hh_day_hour[['day', 'elecheat']]
#elecheat_hh.name = 'elecheat'

#stove_hh = hh_day_hour[['day', 'stove']]
#stove_hh.name = 'stove_hh'

In [106]:
df_list = [
refrigerator_hh,
dishwasher_hh,
kitchen_hh,
lighting_hh,
washer_hh,
microwave_hh]

In [108]:
to_csv(df_list, house_num) #for heatmap

In [ ]:
#stacked - data/    day_by_day
#multiseries - data/house_{0}/dayhour
#heatmap - data/house_{0}/hour_by_hour/ 

In [142]:
!ls data/house_1/hour_by_hour/

dishwasher_hh.csv   lighting_hh.csv     refrigerator_hh.csv
kitchen_hh.csv      microwave_hh.csv    washer_hh.csv


In [106]:
stats = day_by_day.describe()

In [133]:
stats = pd.DataFrame()

In [143]:
stats = refrigerator_hh.describe() + dishwasher_hh.describe() + kitchen_hh.describe() + lighting_hh.describe() + washer_hh.describe() + microwave_hh.describe()

In [156]:
stats = pd.concat([df.describe() for df in df_list], axis=1)

In [157]:
stats

,day,refrigerator,day,dish,day,kitchen,day,lighting,day,washer,day,microwave
count,871.000000,803.000000,871.000000,803.000000,871.000000,8.030000e+02,871.000000,803.000000,871.000000,8.030000e+02,871.000000,803.000000
mean,19.185993,52679.383562,19.185993,22723.099626,19.185993,6.348367e+04,19.185993,71588.022416,19.185993,3.761713e+04,19.185993,19526.740971
std,10.487753,30534.957404,10.487753,101570.953402,10.487753,8.815885e+04,10.487753,56064.853462,10.487753,1.928948e+05,10.487753,51484.637902
min,1.000000,798.000000,1.000000,0.000000,1.000000,7.830000e+02,1.000000,53.000000,1.000000,0.000000e+00,1.000000,68.000000
25%,10.000000,38913.000000,10.000000,17.000000,10.000000,4.051050e+04,10.000000,33701.000000,10.000000,0.000000e+00,10.000000,3736.000000
50%,19.000000,50881.000000,19.000000,33.000000,19.000000,4.172200e+04,19.000000,76279.000000,19.000000,0.000000e+00,19.000000,3800.000000
75%,28.000000,63597.500000,28.000000,68.000000,28.000000,5.396300e+04,28.000000,102183.500000,28.000000,5.750000e+00,28.000000,4459.500000
max,37.000000,204278.000000,37.000000,714493.000000,37.000000,1.581727e+06,37.000000,327617.000000,37.000000,1.857414e+06,37.000000,900333.000000


In [158]:
max_values = [204278.000000, 714493.000000, 1.581727e+06, 327617.000000, 1.857414e+06, 900333.000000]

In [160]:
min(max_values[1:]) #327617.0

327617.0

In [165]:
for i in range(10):
    print(750 + i*35000)

750
35750
70750
105750
140750
175750
210750
245750
280750
315750


In [109]:
basedate = datetime(2011, 4, 18, 0, 0, 0)
otherdate = datetime(2011, 5, 11, 0, 0, 0)
print(otherdate - basedate)

23 days, 0:00:00


In [115]:
basedate = datetime(2011, 4, 18, 0, 0, 0)
otherdate = datetime(2011, 4, 22, 0, 0, 0)
print(otherdate - basedate)

4 days, 0:00:00


In [112]:
17, 18

(17, 18)